# Título Placeholder

## Preparar ambiente

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

## Dados do Sistema e Constantes

In [ ]:
# Tempo em minutos, de acordo com seção 3.3.2. (FABRICAÇÃO DO PRODUTO)
t_total = 60
# Massa de água e mosto, em kg, baseado na seção 3.3.2. (FABRICAÇÃO DO PRODUTO)
massa_agua = 25.5
massa_malte = 8.5
massa_mosto = massa_agua + massa_malte
# Aproximado da capacidade térmica da água em KJ/(kg·K), baseado em https://www.fq.pt/tabelas/capacidades-termicas-massicas
c = 4.1855
# Constantes da equação de Antoine para a água de acordo com http://paginapessoal.utfpr.edu.br/lisandra/fisico-quimica-alimentos/anexos/C8-6_-_Pv_Antoine.pdf/view
ant_A = 18.3036
ant_B = 3816.44
ant_C = -46.13

# Limites baseados nas restrições para a aplicação da Lei de Atoine de acordo com http://paginapessoal.utfpr.edu.br/lisandra/fisico-quimica-alimentos/anexos/C8-6_-_Pv_Antoine.pdf/view
# 284 e 441 Em Kelvin
# Limites reavaliados já que os dados relacinados à equação de Arrhenius forão testados entre 363.15 e 403.15 k (90 C° e 130 C°)
limites_temperatura = [(363.15, 441)]
T_min, T_max = limites_temperatura[0]
# 293.15 k é equivalente a 20 C°, eu achei que seria um valor inicial adequado
T0 = 293.15

# Funções Físico-Químicas

In [ ]:
def isomerizacao(T):
    # "Velocidade da isomerização"
    # A constante de Arrhenius para esse caso seria k = (7.9 × 10^11) e^(-11858/T), on T é a Temperatura Absoluta, baseado em https://doi.org/10.1021/jf0481296
    return 7.9e11 * np.exp(-11858 / T)

def pressao_vapor(T):
    T_celsius = T - 273.15
    return 10 ** (ant_A - ant_B / (ant_C + T_celsius))

def energia_termica_total(T, T0, massa, capacidade_termica, T_min, T_max):
    
    T = np.clip(T, T_min, T_max)
    T = np.insert(T, 0, T0)
    # As diferenças entre as temperaturas do vetor
    delta_T = np.diff(T)
    return np.sum(delta_T * massa * capacidade_termica)

# Função Objetivo

A entrada de temperatura (ou seja, os valores de T) precisa ter espaços de tempo equivalentes.

In [ ]:

def func_objetivo(q):

    
    energia_total = energia_termica_total(T, T0, massa_mosto, c, T_min, T_max)

    # Pesos, de acordo com o objetivo (Minimizar energia e evaporação, maximizar isomerização e regular a densidade), seção 3.3.3.(CONTROLE ÓTIMO MULTIOBJETIVO UTILIZANDO ALGORITMO GENÉTICO)
    w1, w2, w3, w4 = 1, -1, 1, -1

    return (
        w1 * energia_total +
        w2 * iso_total +
        w3 * massa_evaporada +
        w4 * densidade
    )

In [ ]:
res = minimize(funcao_objetivo, calor_inicial, bounds=limites_calor)
print("Resultado da otimização:")
print("Calor aplicado:", res.x[0])
print("Valor da função objetivo:", res.fun)

In [ ]:
plt.bar(['Calor Aplicado'], [res.x[0]], color='blue')
plt.ylabel('Valor Otimizado')
plt.title('Resultado da Otimização do Calor Aplicado')
plt.show()